# model for the domain adaptation:
the first model is for the domains and the second for the dust forecasting

first try with labels:

In [ ]:
class cnn_encoder(nn.Module):
    def __init__(self,in_channels,n,output_linear):
        super(cnn_encoder, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=in_channels, out_channels=n, kernel_size=3, stride=1, padding=1)
        self.bn0 = nn.BatchNorm2d(n)  # BatchNorm after the second convolution
        self.relu0 = nn.ReLU()
        self.resnet_block0 = ResNetBlock(n, n)
        self.pool0 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=n, out_channels=2*n, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(2*n)  # BatchNorm after the first convolution
        self.relu = nn.ReLU()# I WIIL APPLY IT AFTER EVERY COV IN THE FORWARD
        self.resnet_block1 = ResNetBlock(2*n, 2*n)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=2*n, out_channels=3*n, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(3*n)  # BatchNorm after the second convolution
        self.resnet_block2 = ResNetBlock(3*n, 3*n)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=3*n, out_channels=4*n, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(4*n)  # BatchNorm after the second convolution
        self.resnet_block3 = ResNetBlock(4*n, 4*n)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv2d(in_channels=4*n, out_channels=5*n, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(5*n)  # BatchNorm after the second convolution
        self.resnet_block4 = ResNetBlock(5*n, 5*n)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv5 = nn.Conv2d(in_channels=5*n, out_channels=6*n, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(6*n)  # BatchNorm after the second convolution
        self.resnet_block5 = ResNetBlock(6*n, 6*n)
        self.pool5 = nn.MaxPool2d(kernel_size=1, stride=1)

        self.dropout1 = nn.Dropout2d(p=0.1)
        self.dropout2 = nn.Dropout2d(p=0.1)

        self.fc_input_size = 6*n * 1 * 2  
        self.fc1 = nn.Linear(self.fc_input_size, output_linear)
        self.relu = nn.ReLU()


        #for the domain classifer:
        self.fc_input_size = 6*n * 1 * 2  
        self.fc3 = nn.Linear(self.fc_input_size ,256 )
        self.relu4 = nn.ReLU()
        self.fc4 = nn.Linear(256, 2)
        
    def forward(self, x, grl_lambda=1.0):
#         x=x.view(-1,153,54,81)
        x = self.pool0(self.resnet_block0(self.bn0(self.relu(self.conv0(x)))))
        x = self.pool1(self.resnet_block1(self.bn1(self.relu(self.conv1(x)))))
        x = self.pool2(self.resnet_block2(self.bn2(self.relu(self.conv2(x)))))
        x = self.pool3(self.resnet_block3(self.bn3(self.relu(self.conv3(x)))))
        x = self.pool4(self.resnet_block4(self.bn4(self.relu(self.conv4(x)))))
        x = self.pool5(self.resnet_block5(self.bn5(self.relu(self.conv5(x)))))
        x=self.dropout2(x)
        x = x.view(-1, self.fc_input_size)

        # Apply gradient reversal
        feat_domain = x #GradientReverse.apply(x, grl_lambda)

        x = self.relu(self.fc1(x))
 

        # Domain classifier part
        feat_domain = self.fc3(feat_domain)
        feat_domain = self.relu4(feat_domain)
        feat_domain = self.fc4(feat_domain)

        return x, feat_domain

In [ ]:
class dust_pred(nn.Module):
    def __init__(self,linear_input,output_channels):
        super(dust_pred, self).__init__()
        self.fc1 = nn.Linear(linear_input, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, output_channels)
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [ ]:
#the trainind func .

In [ ]:
def train_model_try2(train_dataloader,num_epoch,model_encoder,model_dust,optimizer,scheduler,criterion_label,criterion_sync_time_s,criterion_sync_time_t):
    model_dust_list=[]
    model_encoder_list=[]
    #list_pre_recall_avg=[]
    domain_t=torch.full((25,), 1).to(device)
    domain_s =  torch.full((25,), 0).to(device)
    dict_model_accuracies={}
    
    for i,epoch in enumerate(range(num_epoch)):
        for x_t,x_s,labels,time in tqdm(train_dataloader):
            x_t,x_s,labels=x_t.to(device),x_s.to(device),labels.to(device)
            optimizer.zero_grad()
            output_dust_t,output_domain_t=model_encoder(x_t)
            output_dust_s,output_domain_s=model_encoder(x_s)
            if x_t.shape==torch.Size([11, 204, 54, 81]) or x_s.shape==torch.Size([11, 204, 54, 81]):
                continue
            labels=labels.unsqueeze(1)
            mask = (x_s.sum([1, 2, 3]) > 0) * (x_t.sum([1, 2, 3])>0)
            mask_valid_input=torch.cat([x_s.sum([1, 2, 3]) > 0 , x_t.sum([1, 2, 3])>0])
            if any(mask):

                domain_loss_s=criterion_sync_time_s(output_domain_s,domain_s)[mask].mean()
                domain_loss_t=criterion_sync_time_t(output_domain_t,domain_t)[mask].mean()
                
            output_cat=torch.cat((output_dust_s,output_dust_t),dim=0)
            output_dust=model_dust(output_cat)
            label_cat=torch.cat((labels,labels),dim=0)
            label_cat=label_cat.squeeze(1)
            loss_dust=(criterion_label(output_dust,label_cat)*(label_cat>0)*mask_valid_input).mean()#mask for x_t.sum()>0
            
            loss_total=1500  *loss_dust -domain_loss_s-domain_loss_t
            loss_total.backward(retain_graph=True)
            optimizer.step()
            
        print(f"Epoch [{epoch+1}/{num_epochs}]-total Loss: {loss_total.item():.4f} ,domain_loss_s: {domain_loss_s.item():.4f},domain_loss_t: {domain_loss_t.item():.4f}, dust_loss:{loss_dust:.4f} ")

        with torch.no_grad():
            true_positive_target = 0
            false_positive_target = 0
            false_negative_target = 0
            true_negative_target = 0
            loss_dust_val=0
            #total_count = 0
            for i,values in enumerate(val_dataloader):
                x_t, x_s, labels,time=values
                x_t, x_s, labels = x_t.to(device), x_s.to(device), labels.to(device)
                output_dust_t,output_domain_t = model_encoder(x_t)
                output = model_dust(output_dust_t)
                loss_dust_val+=(criterion_label(output,labels)*(labels>0)).mean()
                _, predicted = torch.max(output.data, 1)
                true_positive_target += ((predicted == labels) & (labels == 2)).sum().item()
                false_positive_target += ((predicted != labels) & (labels == 1)).sum().item()
                false_negative_target += ((predicted != labels) & (labels == 2)).sum().item()
                true_negative_target += ((predicted == labels) & (labels == 1)).sum().item()
                #total_count += labels.size(0)

            loss_dust_val=loss_dust_val/(i+1)
            print(f'true_pos: {true_positive_target},true_neg: {true_negative_target},false_pos:{false_positive_target},false_neg:{false_negative_target}')
            if((true_positive_target + false_positive_target)):
                precision_target = 100 * true_positive_target / (true_positive_target + false_positive_target)  # precision
            else:
                precision_target=torch.tensor([float('nan')])
            recall=  100 * true_positive_target / (true_positive_target + false_negative_target)
            accuracy=100 * (true_positive_target+true_negative_target) / (true_positive_target + false_negative_target+ true_negative_target + false_positive_target)
            recall_precision_avg=((recall+precision_target)/2)
            print(f'acc: {accuracy},precision: {precision_target}, recall: {recall}, recall_pre_avg: {recall_precision_avg}')
            print()
            #print(torch.cuda.memory_summary())
            scheduler.step(accuracy)
            print("Learning rate after:", optimizer.param_groups[0]['lr'])
            

second try , with mse loss between the domains: